In [1]:
!pip install langchain_chroma langchain_community langchain_text_splitters unstructured sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.3/122.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 1

In [2]:
import langchain_chroma
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.directory import DirectoryLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import zipfile
from tqdm import tqdm
import pickle
from torch.utils.data import Dataset, DataLoader

In [3]:
with zipfile.ZipFile('/content/dev_src.zip', 'r') as zip_ref:
    zip_ref.extractall()
with zipfile.ZipFile('/content/dev_susp.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [9]:
# load the document and split it into chunks
loader = DirectoryLoader("/content/dev_src")
documents = loader.load()

In [10]:
for each in documents:
  each.page_content = each.page_content.replace('\n\n', '.')
  each.page_content = each.page_content.replace('\n', '.')
  each.page_content = each.page_content.replace('\t', '.')

In [11]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
# split it into chunks
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0, separators=['.', '?', '!'])
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0, separator='.')
docs = text_splitter.split_documents(documents)

In [15]:
docs[0]

Document(page_content='Главная Биографии Искусство Писатели.Биография Пушкина.Краткие содержания Краткое содержание «Капитанская дочка» Краткое содержание «Дубровский».Презентация', metadata={'source': '/content/dev_src/6528.txt'})

In [16]:
docs[200].page_content

'Клод Поэр основал GEIPAN в 1977 году и стал первым руководителем группы'

In [17]:
len(docs)

24085

In [18]:
# docs

In [19]:
model_name = "ai-forever/sbert_large_nlu_ru"
model_kwargs = {'device': 'cuda'}
embedding_function = SentenceTransformerEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [30]:
db = False
with tqdm(total=len(docs), desc="Ingesting documents") as pbar:
    for d in docs:
      if db:
          db.add_documents([d])
      else:
          db = Chroma.from_documents([d], embedding_function, persist_directory='./Croma_db')
      pbar.update(1)

Ingesting documents: 100%|██████████| 24085/24085 [14:07<00:00, 28.40it/s]


In [21]:
import os as os

In [32]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = 's-nlp/ruRoberta-large-paraphrase-v1'
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [39]:
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to('cuda')
    with torch.inference_mode():
        proba = model(**batch).logits
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
    return proba[0] # p(non-paraphrase), p(paraphrase)

In [40]:
dict_results = {}

In [62]:
pathdir = "/content/dev_susp"
for zfile in os.listdir(path=pathdir):
  print(zfile)
  tloader = TextLoader(pathdir + '/' + zfile)
  text = tloader.load()
  text[0].page_content = text[0].page_content.replace('\n\n', '.')
  text[0].page_content = text[0].page_content.replace('\n', '.')
  text[0].page_content = text[0].page_content.replace('\t', '.')
  text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0, separator=".")
  inputs = text_splitter.split_documents(text)
  outputs = []
  for each in inputs:
    # print(each.page_content)
    outputs.append(db.similarity_search(each.page_content))
  result = set()
  text1 = []
  text2 = []
  counter = 0
  t = max(len(outputs), len(inputs))
  for doc, input in tqdm(zip(outputs, inputs), total=t):
    # добавить сюда создание датафрейма и связь между ними
    for each in doc:
      r = compare_texts(str(input.page_content), str(each.page_content))
      if r[1] > 0.5:
        counter+=1
        text1.append(input.page_content)
        text2.append(each.page_content)
        result.add(each.metadata['source'][17:])
  dict_results[zfile] = {'result': result, 'texts1': text1, 'texts2': text2, 'counter':counter}

017.txt


100%|██████████| 115/115 [00:22<00:00,  5.00it/s]


215.txt


100%|██████████| 111/111 [00:21<00:00,  5.09it/s]


216.txt


...

100%|██████████| 126/126 [00:25<00:00,  5.02it/s]


067.txt


100%|██████████| 104/104 [00:20<00:00,  5.18it/s]


180.txt


100%|██████████| 136/136 [00:25<00:00,  5.41it/s]


In [63]:
for key, value in dict_results.items():
    print(key, ": ", value['result'])
    break

017.txt :  {'1089.txt', '1091.txt', '1090.txt', '1092.txt', '1088.txt'}


In [64]:
list(dict_results['102.txt']['result'])[0][17:]

''

In [65]:
dict_results_cpy = dict_results.copy()

In [66]:
for key, value in dict_results_cpy.items():
  dict_results_cpy[key] = list(dict_results_cpy[key]['result'])

In [67]:
for key, value in dict_results_cpy.items():
  l = []
  for each in value:
    l.append(each)
  dict_results_cpy[key] = l

In [68]:
dict_results_cpy

{'017.txt': ['1089.txt', '1091.txt', '1090.txt', '1092.txt', '1088.txt'],
 '215.txt': ['13764.txt', '13761.txt', '13760.txt', '13762.txt'],
 '216.txt': ['13827.txt', '13825.txt', '13828.txt', '13824.txt', '13826.txt'],
 '223.txt': ['14275.txt', '14273.txt', '14276.txt', '14274.txt', '14272.txt'],
 '195.txt': ['12481.txt',
  '12480.txt',
  '12484.txt',
  '12483.txt',
  '12482.txt',
  '12485.txt'],
 '176.txt': ['11265.txt', '11267.txt', '11268.txt', '11264.txt', '11266.txt'],
 '089.txt': [],
 '248.txt': ['15875.txt', '15874.txt', '15872.txt', '15876.txt', '15873.txt'],
 '265.txt': ['16963.txt', '16960.txt', '16961.txt', '16962.txt'],
 '151.txt': ['9669.txt', '9667.txt', '9666.txt', '9668.txt', '9665.txt'],
 '090.txt': ['5760.txt', '5761.txt', '5764.txt', '5763.txt', '5762.txt'],
 '102.txt': ['6531.txt', '6530.txt', '6529.txt'],
 '154.txt': ['9860.txt', '9857.txt', '9862.txt', '9859.txt', '9864.txt'],
 '057.txt': ['3649.txt', '3652.txt', '3648.txt', '3650.txt', '3651.txt'],
 '050.txt': ['

In [61]:
pairs_dict = {}

with open('/content/dev_pairs.txt', 'r') as file:
    for line in file:
        first_file, second_file = line.strip().split()
        if first_file not in pairs_dict:
            pairs_dict[first_file] = []
        pairs_dict[first_file].append(second_file)

print(pairs_dict)

{'017.txt': ['1091.txt', '1092.txt', '1090.txt', '1088.txt', '1089.txt'], '107.txt': ['6850.txt', '6848.txt', '6849.txt', '6851.txt'], '070.txt': ['4482.txt', '4480.txt', '4481.txt', '4483.txt', '4484.txt'], '216.txt': ['13827.txt', '13826.txt', '13825.txt', '13824.txt', '13828.txt'], '143.txt': ['9155.txt', '9153.txt', '9152.txt', '9154.txt', '9156.txt'], '173.txt': ['11073.txt', '11075.txt', '11074.txt', '11076.txt', '11072.txt'], '242.txt': ['15490.txt', '15489.txt', '15493.txt', '15492.txt', '15488.txt'], '054.txt': ['3460.txt', '3458.txt', '3457.txt', '3456.txt', '3459.txt'], '050.txt': ['3201.txt', '3203.txt', '3204.txt', '3202.txt', '3200.txt', '3205.txt'], '154.txt': ['9861.txt', '9860.txt', '9858.txt', '9864.txt', '9857.txt', '9859.txt', '9863.txt', '9862.txt', '9856.txt'], '151.txt': ['9668.txt', '9667.txt', '9665.txt', '9666.txt', '9669.txt', '9664.txt'], '081.txt': ['5186.txt', '5184.txt', '5185.txt', '5187.txt', '5188.txt'], '183.txt': ['11716.txt', '11718.txt', '11717.txt

In [69]:
# roberta - модель + эмбеддинги sbert
true_positive = 0
predicted_total = 0
original_total = 0

for key in pairs_dict.keys():
    original_values = set(pairs_dict[key])
    predicted_values = set(dict_results_cpy[key])

    true_positive += len(original_values.intersection(predicted_values))
    predicted_total += len(predicted_values)
    original_total += len(original_values)

precision = true_positive / predicted_total
recall = true_positive / original_total

print("Precision:", precision)
print("Recall:", recall)

Precision: 1.0
Recall: 0.75


In [ ]:
# Вторая модель + эмбеддинги e5
true_positive = 0
predicted_total = 0
original_total = 0

for key in pairs_dict.keys():
    original_values = set(pairs_dict[key])
    predicted_values = set(dict_results_cpy[key])

    true_positive += len(original_values.intersection(predicted_values))
    predicted_total += len(predicted_values)
    original_total += len(original_values)

precision = true_positive / predicted_total
recall = true_positive / original_total

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.6133333333333333
Recall: 0.8214285714285714


In [ ]:
# роберта - модель, e5 embs
true_positive = 0
predicted_total = 0
original_total = 0

for key in pairs_dict.keys():
    original_values = set(pairs_dict[key])
    predicted_values = set(dict_results_cpy[key])

    true_positive += len(original_values.intersection(predicted_values))
    predicted_total += len(predicted_values)
    original_total += len(original_values)

precision = true_positive / predicted_total
recall = true_positive / original_total

print("Precision:", precision)
print("Recall:", recall)

Precision: 0.9844961240310077
Recall: 0.7559523809523809


------------------------------

In [ ]:
tloader = TextLoader("/content/011.txt")
text = tloader.load()

In [ ]:
text[0].page_content = text[0].page_content.replace('\n\n', ' ')
text[0].page_content = text[0].page_content.replace('\n', ' ')
text[0].page_content = text[0].page_content.replace('\t', ' ')

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0, separator=" ")
inputs = text_splitter.split_documents(text)
# inputs = tloader.load_and_split()

In [ ]:
len(inputs)

107

In [ ]:
for doc in inputs:
  doc.page_content = "query:"+doc.page_content;

In [ ]:
# query it
outputs = []
for each in inputs:
  # print(each.page_content)
  outputs.append(db.similarity_search(each.page_content))

In [ ]:
# print results
print(outputs[2][2].page_content)

query:гг. до н. э. —/первые дошедшие до нас математические тексты: два египетских папируса и многочисленные глиняные таблички из древнего Вавилона, содержащие формулировки и решения задач. Египтяне


In [ ]:
for doc in outputs:
  for each in doc:
    each.page_content = each.page_content[6:]

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = 's-nlp/ruRoberta-large-paraphrase-v1'
model = AutoModelForSequenceClassification.from_pretrained(model_name).cpu()
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
    return proba[0] # p(non-paraphrase), p(paraphrase)

In [ ]:
def get_similarity(text1, text2):
    """ Predict the probability that two Russian sentences are paraphrases of each other. """
    with torch.inference_mode():
        text = text1+"</s>" + text2
        batch = tokenizer(
            text,
            truncation=True, max_length=model.config.max_position_embeddings, return_tensors='pt',
        ).to(model.device)
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0][1].item()

In [ ]:
def get_similarity2(text1, text2):
    """ Predict the probability that two Russian sentences are paraphrases of each other. """
    with torch.inference_mode():
        batch = tokenizer(
            text1, text2,
            truncation=True, max_length=model.config.max_position_embeddings, return_tensors='pt',
        ).to(model.device)
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0][1].item()

In [ ]:
for doc in inputs:
    doc.page_content = doc.page_content[6:]

In [ ]:
from tqdm import tqdm

In [ ]:
result = set()
text1 = []
text2 = []
counter = 0
t = max(len(outputs), len(inputs))
for doc, input in tqdm(zip(outputs, inputs), total=t):
  for each in doc:
    r = compare_texts(str(input.page_content), str(each.page_content))
    if r[1] > 0.5:
      counter+=1
      text1.append(input.page_content)
      text2.append(each.page_content)
      result.add(each.metadata['source'])

100%|██████████| 107/107 [10:43<00:00,  6.02s/it]


In [ ]:
result

{'/content/dev_src/704.txt', '/content/dev_src/705.txt'}

In [ ]:
counter

14

In [ ]:
text1[4]

'математики вклад римлян был незначителен. Римская система счисления основывалась на громоздких обозначениях чисел. В целом, римляне только на словах были знатоками, а на деле они ничего не создали'

In [ ]:
text2[4]

'практике, извлекая из них реальную пользу. Однако в развитие самой математики вклад римлян был незначителен. Римская система счисления основывалась на громоздких обозначениях чисел. Главной ее'

In [ ]:
borrowed_text = set(text1)
' '.join(borrowed_text)

'развитие теория дифференциальных уравнений, дифференциальная геометрия. В ХІХи XX вв. математика поднимается на новый уровень абстракции. Обычные величины и числа оказываются лишь частными случаями объектов, изучаемых современной алгеброй. А геометрия переходит к исследованию пространств и пространственных отношений. Развиваются новые дисциплины, такие как теория функций комплексного математики вклад римлян был незначителен. Римская система счисления основывалась на громоздких обозначениях чисел. В целом, римляне только на словах были знатоками, а на деле они ничего не создали областей науки, проникновение математических методов во многие сферы человеческой деятельности, быстрый прогресс вычислительной техники. Эти тенденции привели к появлению целого ряда новых математического анализа, он находил все более широкое применение. Дифференциальное и интегральное исчисления стали краеугольными камнями математического анализа. Со временем математический анализ предполагая, что при движении 

In [ ]:
docs

[Document(page_content='оказалось и несколько ошибочных доказательств. Создание новых алгебр, начавшееся с квартернионов, породило аналогичные сомнения и в', metadata={'source': '/content/dev_src/704.txt'}),
 Document(page_content='существования в евклидовой геометрии логических пробелов. Одним из недостатков евклидовых Начал было использование допущений, не', metadata={'source': '/content/dev_src/704.txt'}),
 Document(page_content='свойством коммутативности, т.е. ab = ba. Кватернионы, совершившие переворот в традиционных представлениях о числах, были открыты в 1843', metadata={'source': '/content/dev_src/704.txt'}),
 Document(page_content='надежность, чем та, которой обладали аксиомы. Неевклидова геометрия и кватернионы (алгебра, в которой не выполняется свойство', metadata={'source': '/content/dev_src/704.txt'})]

In [ ]:
text1[0]

'двух в виде отношения двух целых чисел, что было крайне важно для их философии.\nВеличины, непредставимые в виде отношения целых чисел, пифагорейцы назвали\nнесоизмеримыми.\nПифагорейцы представляли'

In [ ]:
text2[0]

'ло крайне важно для их философии. Величины, непредставимые в виде отношения целых чисел, пифагорейцы назвали несоизмеримыми;'